In [1]:
import pandas as pd

df = pd.read_csv('data/Nifty50Scrips.csv')

stock_ids = df['Symbol']


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 100:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-s:test_list.index('Sep 2021')+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]
    
    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-5)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

POWERGRID
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11

[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
BHARTIARTL
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*****

[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
CIPLA
162
2019-02-15 00:00:00
[**********

[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
KOTAKBANK
188
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[******

[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
ITC
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[************

[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
HDFCBANK
164
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*******

[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
AXISBANK
164
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*******

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
6,POWERGRID.NS,2020-08-15,131.55,2.94,9457.0,15.21,38020.0,8.65
7,POWERGRID.NS,2020-11-15,141.00,4.44,9530.0,15.96,38499.0,8.83
8,POWERGRID.NS,2021-02-15,159.49,4.83,10142.0,16.96,39277.0,9.40
9,POWERGRID.NS,2021-05-16,171.64,5.06,10510.0,17.27,39639.0,9.94
10,POWERGRID.NS,2021-08-15,184.75,8.60,10216.0,22.93,40398.0,8.06
...,...,...,...,...,...,...,...,...
7,ASIANPAINT.NS,2020-11-15,2184.60,8.66,5350.0,23.73,18329.0,92.06
8,ASIANPAINT.NS,2021-02-15,2486.10,12.91,6788.0,28.67,19697.0,86.71
9,ASIANPAINT.NS,2021-05-16,2774.50,8.88,6651.0,32.73,21712.0,84.77
10,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15


In [3]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(TTM)'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS'])*100/tmp['Prev_EPS'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['SG%'] = round((tmp['Sales(TTM)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

<ipython-input-3-4e6773986fa5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS'] = tmp['EPS(TTM)'].shift(1)
<ipython-input-3-4e6773986fa5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
<ipython-input-3-4e6773986fa5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS,Prev_PE,Prev_Close,Prev_Sales,FR%,SR%,TR%,SG%
6,POWERGRID.NS,2020-08-15,131.55,2.94,9457.0,15.21,38020.0,8.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,POWERGRID.NS,2020-11-15,141.00,4.44,9530.0,15.96,38499.0,8.83,15.21,8.65,131.55,38020.0,4.93,2.18,7.11,1.26
8,POWERGRID.NS,2021-02-15,159.49,4.83,10142.0,16.96,39277.0,9.40,15.96,8.83,141.00,38499.0,6.27,6.86,13.13,2.02
9,POWERGRID.NS,2021-05-16,171.64,5.06,10510.0,17.27,39639.0,9.94,16.96,9.40,159.49,39277.0,1.83,5.85,7.68,0.92
10,POWERGRID.NS,2021-08-15,184.75,8.60,10216.0,22.93,40398.0,8.06,17.27,9.94,171.64,39639.0,32.77,-25.12,7.65,1.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,ASIANPAINT.NS,2020-11-15,2184.60,8.66,5350.0,23.73,18329.0,92.06,23.65,76.22,1802.55,18030.0,0.34,20.85,21.19,1.66
8,ASIANPAINT.NS,2021-02-15,2486.10,12.91,6788.0,28.67,19697.0,86.71,23.73,92.06,2184.60,18329.0,20.82,-7.02,13.80,7.46
9,ASIANPAINT.NS,2021-05-16,2774.50,8.88,6651.0,32.73,21712.0,84.77,28.67,86.71,2486.10,19697.0,14.16,-2.55,11.61,10.23
10,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15,32.73,84.77,2774.50,21712.0,11.15,-3.44,7.71,12.26


In [4]:
df_analysis_final[df_analysis_final['Quarter'] == '2021-08-15'].sort_values(by='FR%', ascending=False)

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS,Prev_PE,Prev_Close,Prev_Sales,FR%,SR%,TR%,SG%
10,TATASTEEL.NS,2021-08-15,1461.65,74.08,53372.0,176.33,184191.0,8.29,63.04,17.96,1132.10,156294.0,179.71,-150.61,29.10,17.85
10,SUNPHARMA.NS,2021-08-15,778.30,6.02,9719.0,25.03,35632.0,31.09,12.11,57.10,691.45,33498.0,106.69,-94.15,12.54,6.37
10,HINDALCO.NS,2021-08-15,442.05,12.40,41358.0,31.06,148060.0,14.23,15.50,24.70,382.80,131985.0,100.39,-84.95,15.44,12.18
10,JSWSTEEL.NS,2021-08-15,749.05,24.42,28902.0,59.47,96959.0,12.60,32.73,21.60,707.10,79839.0,81.70,-75.69,6.01,21.44
10,ONGC.NS,2021-08-15,116.10,4.76,108136.0,17.59,406212.0,6.60,12.92,8.74,112.95,360572.0,36.15,-33.33,2.82,12.66
10,GRASIM.NS,2021-08-15,1496.90,25.34,19919.0,87.21,83268.0,17.16,65.43,20.89,1367.00,76393.0,33.29,-23.80,9.49,9.00
10,POWERGRID.NS,2021-08-15,184.75,8.60,10216.0,22.93,40398.0,8.06,17.27,9.94,171.64,39639.0,32.77,-25.12,7.65,1.91
10,TITAN.NS,2021-08-15,1837.90,0.23,3473.0,14.46,23139.0,127.10,10.95,132.95,1455.85,21645.0,32.05,-5.81,26.24,6.90
10,M&M.NS,2021-08-15,778.65,3.41,19172.0,18.78,81481.0,41.46,14.58,50.86,741.50,74278.0,28.81,-23.81,5.00,9.70
10,EICHERMOT.NS,2021-08-15,2546.80,8.67,1974.0,59.98,9876.0,42.46,49.29,49.00,2415.20,8720.0,21.69,-16.24,5.45,13.26


In [5]:
df_analysis_final[df_analysis_final['Quarter'] == '2021-11-15'].sort_values(by='SG%', ascending=False)

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS,Prev_PE,Prev_Close,Prev_Sales,FR%,SR%,TR%,SG%
11,JSWSTEEL.NS,2021-11-15,671.35,29.66,32503.0,82.54,110198.0,8.13,59.47,12.60,749.05,96959.0,38.79,-49.26,-10.47,13.65
11,TITAN.NS,2021-11-15,2539.50,7.20,7493.0,19.69,26079.0,128.97,14.46,127.10,1837.90,23139.0,36.17,2.00,38.17,12.71
11,IOC.NS,2021-11-15,132.95,6.59,133202.0,27.30,470421.0,4.87,27.11,3.86,104.65,418928.0,0.70,26.35,27.05,12.29
11,TATASTEEL.NS,2021-11-15,1287.25,99.07,60283.0,261.50,205534.0,4.92,176.33,8.29,1461.65,184191.0,48.30,-60.29,-11.99,11.59
11,HINDALCO.NS,2021-11-15,468.55,15.21,47665.0,44.55,164488.0,10.52,31.06,14.23,442.05,148060.0,43.43,-37.39,6.04,11.10
11,RELIANCE.NS,2021-11-15,2593.10,21.58,167611.0,82.47,574995.0,31.44,75.98,28.24,2145.65,518620.0,8.54,12.30,20.84,10.87
11,BPCL.NS,2021-11-15,427.70,14.76,76439.0,82.71,290140.0,5.17,78.38,5.80,454.90,263110.0,5.52,-11.45,-5.93,10.27
11,ONGC.NS,2021-11-15,154.65,14.35,122029.0,28.59,444622.0,5.41,17.59,6.60,116.10,406212.0,62.54,-29.30,33.24,9.46
11,SBILIFE.NS,2021-11-15,1174.90,2.47,25636.0,12.35,88647.0,95.13,12.88,88.40,1138.60,81651.0,-4.11,7.30,3.19,8.57
11,ASIANPAINT.NS,2021-11-15,3122.90,6.21,7096.0,33.93,26120.0,92.04,36.38,82.15,2988.70,24374.0,-6.73,11.23,4.50,7.16


In [6]:
df_analysis_final.to_csv('data/Nifty_FvS_QoQ_Analysis.csv', index=False)